In [ ]:
#Importer les modules:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
import colorama
from colorama import Fore
from colorama import Style
import os, sys
import matplotlib
from matplotlib import cm
from matplotlib.patches import Circle, Wedge, Rectangle
from math import log10

import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols


import ast
import scipy.stats as stats
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.inspection import permutation_importance
import math
from datetime import datetime

In [2]:
print(pd.__version__)

1.3.4


In [ ]:
#A. IMPORTER LES DONNÉES:

In [3]:
cd Desktop\Data Science candidature\Projets\Projet 4\Data\

C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 4\Data


In [4]:
ls

 Le volume dans le lecteur C s'appelle OS
 Le num‚ro de s‚rie du volume est 609B-8774

 R‚pertoire de C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 4\Data

12/06/2022  11:32    <DIR>          .
12/06/2022  11:32    <DIR>          ..
01/07/2021  00:01         1ÿ588ÿ303 2015-building-energy-benchmarking.csv
01/07/2021  00:01         1ÿ235ÿ434 2016-building-energy-benchmarking.csv
26/03/2022  18:13           770ÿ246 archive.zip
14/06/2022  00:22           391ÿ478 Data_clean.csv
14/06/2022  00:22           466ÿ450 Data_clean_bins.csv
14/06/2022  00:22           271ÿ956 Data_clean_E.csv
01/07/2021  00:01            54ÿ630 socrata_metadata_2015-building-energy-benchmarking.json
01/07/2021  00:01            45ÿ816 socrata_metadata_2016-building-energy-benchmarking.json
               8 fichier(s)        4ÿ824ÿ313 octets
               2 R‚p(s)  152ÿ564ÿ285ÿ440 octets libres


In [5]:
df=pd.read_csv('Data_clean.csv', sep=',', low_memory=False)

In [6]:
df.count()

OSEBuildingID                    3247
TaxParcelIdentificationNumber    3246
PropertyName                     3247
SourceEUI_kBtu_sf_log            3247
TotalGHGEmissions_log            3247
BuildingType                     3247
YearBuilt                        3247
PropertyGFATotal                 3247
Neighborhood                     3247
ZipCode_New                      3247
dtype: int64

In [7]:
df.dtypes

OSEBuildingID                      int64
TaxParcelIdentificationNumber     object
PropertyName                      object
SourceEUI_kBtu_sf_log            float64
TotalGHGEmissions_log            float64
BuildingType                      object
YearBuilt                          int64
PropertyGFATotal                   int64
Neighborhood                      object
ZipCode_New                        int64
dtype: object

In [8]:
df.head(5)

,OSEBuildingID,TaxParcelIdentificationNumber,PropertyName,SourceEUI_kBtu_sf_log,TotalGHGEmissions_log,BuildingType,YearBuilt,PropertyGFATotal,Neighborhood,ZipCode_New
0,1,659000030,MAYFLOWER PARK HOTEL,5.161925,5.523179,NonResidential,1927,88434,DOWNTOWN,98101
1,2,659000220,PARAMOUNT HOTEL,5.259057,5.577879,NonResidential,1996,103566,DOWNTOWN,98101
2,3,659000475,WESTIN HOTEL,5.495938,7.631664,NonResidential,1969,961990,DOWNTOWN,98101
3,8,659000970,WARWICK SEATTLE HOTEL,5.437209,6.231858,NonResidential,1980,119890,DOWNTOWN,98121
4,9,660000560,WEST PRECINCT (SEATTLE POLICE),5.750984,5.722343,Nonresidential COS,1999,97288,DOWNTOWN,98101


In [ ]:
#On a, parmi les variables descriptives 2 variables catégoriques: 'BuildingType' et 'Neighborhood', il faut les rendre
#continues en utilisant OneHotEncoder.

#B. OneHotEncoder:

#B.1. On commence par lancer le OneHotEncoder dans les variables catégorielles.

In [9]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)

In [ ]:
#B.1.1 'BuildingType':

In [10]:
encoder_BuildingType = pd.DataFrame(encoder.fit_transform(df[['BuildingType']]))
encoder_BuildingType

,0,1,2,3,4
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
3242,0.0,1.0,0.0,0.0,0.0
3243,0.0,1.0,0.0,0.0,0.0
3244,0.0,1.0,0.0,0.0,0.0
3245,0.0,1.0,0.0,0.0,0.0


In [ ]:
#On renomme les variables catégorielles pour pouvoir mieux se retrouver.
#Notez que OneHotEncoder prend en considération les différentes valeurs d'une colonne par ordre alphabétique.

In [11]:
sorted(df['BuildingType'].unique())

['Campus',
 'NonResidential',
 'Nonresidential COS',
 'Nonresidential WA',
 'SPS-District K-12']

In [12]:
encoder_BuildingType.rename(columns={0:'Campus',1:'Multifamily_HR',2:'Multifamily_LR',3:'Multifamily_MR', 4:'NonResidential', 5:'NonResidential_COS', 6:'NonResidential_WA', 7:'SPS-District_K-12'},inplace = True)
encoder_BuildingType

,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,NonResidential
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
3242,0.0,1.0,0.0,0.0,0.0
3243,0.0,1.0,0.0,0.0,0.0
3244,0.0,1.0,0.0,0.0,0.0
3245,0.0,1.0,0.0,0.0,0.0


In [ ]:
#Vérification.

In [13]:
encoder_BuildingType.sum()

Campus              46.0
Multifamily_HR    2874.0
Multifamily_LR     136.0
Multifamily_MR       1.0
NonResidential     190.0
dtype: float64

In [ ]:
#On retrouve bien le nombre de bâtiments par catégorie qu'on avait lors de l'analyse exploratoire.

#B.1.2 'Neighborhood':

In [14]:
encoder_Neighborhood = pd.DataFrame(encoder.fit_transform(df[['Neighborhood']]))
encoder_Neighborhood

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3244,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3245,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#On renomme les variables catégorielles pour pouvoir mieux se retrouver.
#Notez que OneHotEncoder prend en considération les différentes valeurs d'une colonne par ordre alphabétique.

In [15]:
sorted(df['Neighborhood'].unique())

['BALLARD',
 'CENTRAL',
 'DELRIDGE',
 'DOWNTOWN',
 'EAST',
 'GREATER DUWAMISH',
 'LAKE UNION',
 'MAGNOLIA / QUEEN ANNE',
 'NORTH',
 'NORTHEAST',
 'NORTHWEST',
 'SOUTHEAST',
 'SOUTHWEST']

In [16]:
encoder_Neighborhood.rename(columns={0:'BALLARD',1:'CENTRAL',2:'DELRIDGE',3:'DOWNTOWN', 4:'EAST', 5:'GREATER_DUWAMISH', 6:'LAKE_UNION', 7:'MAGNOLIA_QUEEN_ANNE', 8:'NORTH', 9:'NORTHEAST', 10:'NORTHWEST', 11:'SOUTHEAST', 12:'SOUTHWEST' },inplace = True)
encoder_Neighborhood

,BALLARD,CENTRAL,DELRIDGE,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3244,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3245,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Vérification.

In [17]:
encoder_Neighborhood.sum()

BALLARD                135.0
CENTRAL                103.0
DELRIDGE                91.0
DOWNTOWN               711.0
EAST                   240.0
GREATER_DUWAMISH       676.0
LAKE_UNION             290.0
MAGNOLIA_QUEEN_ANNE    292.0
NORTH                  129.0
NORTHEAST              236.0
NORTHWEST              173.0
SOUTHEAST               91.0
SOUTHWEST               80.0
dtype: float64

In [ ]:
#On retrouve bien le nombre de bâtiments par catégorie qu'on avait lors de l'analyse exploratoire.
#--------------------------------------------------------------------------------------------------------------------------

#B.2 On fusionne les colonnes résultant du OneHotEncoder dans le tableau initial.

In [18]:
f1_df = df.join(encoder_BuildingType)

In [19]:
f2_df=df.join(encoder_Neighborhood)

In [20]:
final_df= f1_df.join(encoder_Neighborhood)
final_df.count()

OSEBuildingID                    3247
TaxParcelIdentificationNumber    3246
PropertyName                     3247
SourceEUI_kBtu_sf_log            3247
TotalGHGEmissions_log            3247
BuildingType                     3247
YearBuilt                        3247
PropertyGFATotal                 3247
Neighborhood                     3247
ZipCode_New                      3247
Campus                           3247
Multifamily_HR                   3247
Multifamily_LR                   3247
Multifamily_MR                   3247
NonResidential                   3247
BALLARD                          3247
CENTRAL                          3247
DELRIDGE                         3247
DOWNTOWN                         3247
EAST                             3247
GREATER_DUWAMISH                 3247
LAKE_UNION                       3247
MAGNOLIA_QUEEN_ANNE              3247
NORTH                            3247
NORTHEAST                        3247
NORTHWEST                        3247
SOUTHEAST   

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#C.Le dataset.

#C.1 Élimination des variables non nécessaires.

#On élimine les colonnes des variables catégorielles initiales et celles d'identification des bâtiments:
#'OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'.
#On reste aussi uniquement avec la variable cible qui nous intéresse: 'TotalGHGEmissions_log'.

In [21]:
final_df.drop({'SourceEUI_kBtu_sf_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#f1_df.drop({'TotalGHGEmissions_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#f2_df.drop({'TotalGHGEmissions_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#C.2 Le dataset.

In [22]:
dataset=final_df
dataset.head(5)

,TotalGHGEmissions_log,YearBuilt,PropertyGFATotal,ZipCode_New,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,NonResidential,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,5.523179,1927,88434,98101,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.577879,1996,103566,98101,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7.631664,1969,961990,98101,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.231858,1980,119890,98121,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.722343,1999,97288,98101,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#C.3 Training et Test sets.

#On divise les données en 2 sets: un "training set" et un "test set".

In [23]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_dataset

,TotalGHGEmissions_log,YearBuilt,PropertyGFATotal,ZipCode_New,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,NonResidential,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
1559,2.859913,1904,72859,98108,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
716,3.641001,1960,20539,98107,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1726,4.619665,2009,89821,98118,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2622,2.760643,1960,36498,98119,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
298,6.164451,2001,378443,98109,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,3.993972,2007,64015,98108,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,4.641695,1985,193500,98121,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2699,3.678071,1952,41308,98134,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1181,3.273364,1920,31332,98116,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
train_dataset.sum()

TotalGHGEmissions_log    1.034574e+04
YearBuilt                5.096048e+06
PropertyGFATotal         3.004196e+08
ZipCode_New              2.549069e+08
Campus                   3.500000e+01
Multifamily_HR           2.303000e+03
Multifamily_LR           1.070000e+02
Multifamily_MR           1.000000e+00
NonResidential           1.520000e+02
BALLARD                  1.080000e+02
CENTRAL                  8.800000e+01
DELRIDGE                 7.100000e+01
DOWNTOWN                 5.870000e+02
EAST                     1.900000e+02
GREATER_DUWAMISH         5.210000e+02
LAKE_UNION               2.340000e+02
MAGNOLIA_QUEEN_ANNE      2.360000e+02
NORTH                    9.500000e+01
NORTHEAST                1.850000e+02
NORTHWEST                1.400000e+02
SOUTHEAST                7.700000e+01
SOUTHWEST                6.600000e+01
dtype: float64

In [ ]:
#C.4 Quelques statistiques.

In [25]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
TotalGHGEmissions_log,2598.0,3.982195,1.440966,-1.609438,3.034712,3.922765,4.936863,9.733410e+00
YearBuilt,2598.0,1961.527329,32.799983,1900.000000,1929.250000,1965.000000,1989.000000,2.015000e+03
PropertyGFATotal,2598.0,115634.953811,262326.888549,16300.000000,30152.000000,50017.000000,104313.250000,9.320156e+06
ZipCode_New,2598.0,98116.581601,17.223689,98006.000000,98104.000000,98109.000000,98122.000000,9.819900e+04
Campus,2598.0,0.013472,0.115306,0.000000,0.000000,0.000000,0.000000,1.000000e+00
Multifamily_HR,2598.0,0.886451,0.317324,0.000000,1.000000,1.000000,1.000000,1.000000e+00
Multifamily_LR,2598.0,0.041186,0.198757,0.000000,0.000000,0.000000,0.000000,1.000000e+00
Multifamily_MR,2598.0,0.000385,0.019619,0.000000,0.000000,0.000000,0.000000,1.000000e+00
NonResidential,2598.0,0.058507,0.234744,0.000000,0.000000,0.000000,0.000000,1.000000e+00
BALLARD,2598.0,0.041570,0.199644,0.000000,0.000000,0.000000,0.000000,1.000000e+00


In [ ]:
#C.5 Normalisation des données.

#Il est recommandé de normaliser les variables qui ont différentes échelles (cf. statistiques en haut). En fait, même si 
#le modèle peut converger sans normalisation ça rend l'entraînement plus difficile et le résultat dépendant des unités 
#utilisées pour les variables.

#Notez que nous avons générer les statistiques précédentes uniquement sur le "set training", ces statistiques sont également
#utilisées pour normaliser l'ensemble de données de test, ceci l'ensemble de données de test doivent être projetées dans la 
#même distribution que celle sur laquelle le training fut fait.

#On a décidé de normaliser aussi la variable cible. En fait même si ceci n'est pas vraiment nécessaire, c'est juste que des 
#fois si on ne normalise pas la variable cible il peut y avoir des bias et on a préféré d'éviter ceci depuis le début.

In [26]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
train_dataset_n = norm(train_dataset)
test_dataset_n = norm(test_dataset)

In [27]:
train_stats_n = train_dataset_n.describe()
train_stats_n = train_stats_n.transpose()
train_stats_n

,count,mean,std,min,25%,50%,75%,max
TotalGHGEmissions_log,2598.0,-2.765131e-15,1.0,-3.880475,-0.657533,-0.041243,0.662520,3.991222
YearBuilt,2598.0,-5.314584e-16,1.0,-1.875834,-0.984065,0.105874,0.837582,1.630265
PropertyGFATotal,2598.0,1.210968e-17,1.0,-0.378669,-0.325864,-0.250138,-0.043159,35.087982
ZipCode_New,2598.0,-3.505173e-13,1.0,-6.420321,-0.730482,-0.440185,0.314590,4.785177
Campus,2598.0,4.441640e-16,1.0,-0.116836,-0.116836,-0.116836,-0.116836,8.555722
Multifamily_HR,2598.0,-8.931355e-18,1.0,-2.793524,0.357833,0.357833,0.357833,0.357833
Multifamily_LR,2598.0,3.694333e-17,1.0,-0.207215,-0.207215,-0.207215,-0.207215,4.824045
Multifamily_MR,2598.0,4.746679e-16,1.0,-0.019619,-0.019619,-0.019619,-0.019619,50.950961
NonResidential,2598.0,1.198896e-16,1.0,-0.249236,-0.249236,-0.249236,-0.249236,4.010725
BALLARD,2598.0,3.077044e-16,1.0,-0.208223,-0.208223,-0.208223,-0.208223,4.800696


In [ ]:
#C.6 Séparez la variable cible des variables prédictives. 

#La variable cible est la valeur que l'on entraînera le modèle à prédire.

In [28]:
train_cible_n = train_dataset_n.pop('TotalGHGEmissions_log')
test_cible_n = test_dataset_n.pop('TotalGHGEmissions_log')

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#D. Les Modèles.
#Construction et entraînement des modèles.
    #Notez que la fonction ".fit", c'est celle qui est chargée d'entraîner notre modèle et que la fonction ".predict", c'est
    #celle qui est chargée de prédire la variable cible à partir du dataset test.
    
    
#------

#D.1 DummyRegressor: 
    #On prend la médiane car elle est statistiquement préférable à la moyenne.
    #Sans optimisation des hyperparamètres (pour le DummyRegressor il n'y a pas vraiment des paramètres à optimiser).

In [29]:
DM= DummyRegressor(strategy='median')

In [ ]:
#D.1.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [30]:
start_time = datetime.now()

In [31]:
DM.fit(train_dataset_n,train_cible_n)

DummyRegressor(strategy='median')

In [32]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [33]:
DM_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [34]:
DM.score(train_dataset_n,train_cible_n)

-0.001701645482618197

In [ ]:
#Notez que la valeur absolue du R^2 (qui est donné par le "score") est loin de 1, ce qui montre la faiblesse de ce 
#modèle.

#D.1.2 Prédiction.

In [35]:
DM_prediction=DM.predict(test_dataset_n)
DM_prediction

array([-0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04124307,
       -0.04124307, -0.04124307, -0.04124307, -0.04124307, -0.04

In [ ]:
#D.1.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [36]:
R2_DM=r2_score(test_cible_n,DM_prediction)
R2_DM

-0.004425104178547912

In [ ]:
#Calcul du MSE:

In [37]:
MSE_DM=mean_squared_error(test_cible_n,DM_prediction)

In [ ]:
#Calcul du RMSE:

In [38]:
RMSE_DM=math.sqrt(MSE_DM)

In [ ]:
#Calcul du MAE:

In [39]:
 MAE_DM=mean_absolute_error(test_cible_n,DM_prediction)

In [ ]:
#D.1.5 Résultats.

In [40]:
results_DM= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_DM= results_DM.append({'Modèle':'Dummy Regressor','R^2':R2_DM,'MSE':MSE_DM,'RMSE':RMSE_DM,'MAE':MAE_DM ,'Temps de calcul (s)':DM_time.seconds},ignore_index=True)
results_DM

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.004425,1.04874,1.02408,0.792805,1


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1 (des valeurs de R^2 proches de 1 montrent quele modèle peut prédire relativement 
#  les données avec précision)
# -la valeur du RMSE est supérieure à 0.5 (des valeurs de RMSE comprises entre 0.2 et 0.5 montrent que le modèle peut 
#  prédire relativement les données avec précision) et
# -La valeur du MAE de modèle naïf de Random forest servira de base, on rappelle qu'il n'y a pas de valeur correcte pour la 
#  MSE. En termes simples, plus la valeur est faible, mieux c'est et 0 signifie que le modèle est parfait. On se base dans
#  la MEA d'un modèle naïf (comme le dummy regressor) pour comparer avec la MEA de modèles plus performants.

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle.


#-----------------------------------------------------------------------

#D.2 LinearRegression.
    #Sans optimisation des hyperparamètres (pour la régression linéaire il n'y a pas vraiment des paramètres à optimiser).

In [41]:
LR=LinearRegression()

In [42]:
LR.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': False,
 'positive': False}

In [ ]:
#D.2.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [43]:
start_time = datetime.now()

In [44]:
LR.fit(train_dataset_n,train_cible_n)

LinearRegression()

In [45]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [46]:
LR_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [47]:
LR.score(train_dataset_n,train_cible_n)

0.22607667788918762

In [ ]:
#D.2.2 Prédiction.

In [48]:
LR_prediction=LR.predict(test_dataset_n)
LR_prediction

array([ 3.79680825e-02,  2.59617669e-01,  1.50844286e-02,  1.54489073e-01,
        5.63491354e-01,  4.91977519e-01,  1.22904492e-01,  1.44286391e+00,
        4.83332960e-01,  4.97876771e-01,  5.54032364e-02, -6.86956866e-02,
        2.85609940e-01,  9.55076820e-01,  5.22690614e-02,  3.59199599e-01,
        1.38881094e-02,  3.39163975e-01, -5.70852010e-01, -1.65924937e-01,
        2.32028940e-01, -4.77692500e-02, -1.47408907e-01,  1.12672917e+00,
        1.44711010e+00,  1.09403398e-01,  1.86083974e+00,  5.38052585e-01,
       -1.52023368e-01,  2.16273582e-01,  1.05719971e-01,  3.50026866e-01,
        1.15802655e+00,  1.59987630e-03, -3.28507649e-02,  6.67030896e-01,
       -9.81648583e-02,  3.18967011e-01,  1.60058572e+00,  7.91296379e-01,
       -2.08754033e-01,  2.64123755e-01, -1.52820362e-01,  1.09269864e-01,
        1.26587912e-01,  1.15436187e+00,  9.16832565e-02,  4.38052713e-01,
        5.89211481e-01,  2.24926960e+00,  8.74016249e-01,  1.02565258e+00,
        1.23042416e+00, -

In [ ]:
#D.2.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [49]:
R2_LR=r2_score(test_cible_n,LR_prediction)
R2_LR

0.2892867907889912

In [ ]:
#Calcul du MSE:

In [50]:
MSE_LR=mean_squared_error(test_cible_n,LR_prediction)

In [ ]:
#Calcul du RMSE:

In [51]:
RMSE_LR=math.sqrt(MSE_LR)

In [ ]:
#Calcul du MAE:

In [52]:
 MAE_LR=mean_absolute_error(test_cible_n,LR_prediction)

In [ ]:
#D.2.5 Résultats.

In [53]:
results_LR= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_LR= results_LR.append({'Modèle':'Linear regression','R^2':R2_LR,'MSE':MSE_LR,'RMSE':RMSE_LR,'MAE':MAE_LR ,'Temps de calcul (s)':LR_time.seconds},ignore_index=True)
results_LR

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Linear regression,0.289287,0.74207,0.861435,0.682741,5


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche aue celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celle du modèle naïf de dummy regressor(ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor.


#D.2.6 Coefficients obtenus par le modèle:

In [54]:
pd.DataFrame(LR.coef_, train_dataset_n.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.154452
PropertyGFATotal,0.331872
ZipCode_New,-0.058317
Campus,0.061201
Multifamily_HR,-0.049520
Multifamily_LR,0.035614
Multifamily_MR,-0.007430
NonResidential,0.007346
BALLARD,-0.024989
CENTRAL,-0.005163


In [ ]:
#-----------------------------------------------------------------------

#D.3 ElasticNet (méthode de régression linéaire).

    #C'est une extension de la régression linéaire, avec un paramètre de pénalité supplémentaire qui vise à minimiser la 
    #complexité et/ou à réduire le nombre de variables (features) utilisées dans le modèle final, ce qui rend intéressant 
    #le fait de le tester. Quoi qu'il en soit, 'objectif -comme pour de nombreux modèles- est de réduire l'erreur de 
    #l'ensemble de test.
    #ElasticNet est un mélange de Lasso (qui inclu la pénalité L1) et Ridge Regression (qui inclue la pénalité L2).

In [55]:
EN=ElasticNet()

In [56]:
EN.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [ ]:
#Le modèle ElasticNet est:
#(1 / (2 * n_samples) )* ||y - Xw||^2_2 + alpha * l1_ratio * ||w||_1 + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

#Les hyperparamètres intéressant à être modifiés sont:

# -alpha:= attribue le poids accordé à chacune des pénalités L1 et L2.

# -l1_ratio:= C'est un paramètre de pénalité de mélange. Pour l1_ratio = 0 la pénalité est une pénalité L2 (Ridge). 
#             Pour l1_ratio = 1 c'est une pénalité L1 (Lasso). Pour 0 < l1_ratio < 1, la pénalité est une combinaison de L1 
#             et L2.

# -tol:= La tolérance d'optimisation, ie. quand le modèle arrête-t-il le processus d'optimisation.



#D.3.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [57]:
start_time = datetime.now()

In [58]:
parameters = {'tol' : [0.1,0.01,0.001,0.0001], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': np.arange(0.0, 1.0, 0.1)} #0: Ridge; 1:Lasso

EN = GridSearchCV(estimator = ElasticNet(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.3.1 Entraînement du modèle.

In [59]:
EN.fit(train_dataset_n,train_cible_n)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 790.9703140319423, tolerance: 203.8066273711174
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 822.9058730554499, tolerance: 211.83662078059308
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 799.6099692363339, tolerance: 202.60153622576405
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 62.197428487466595, tolerance: 0.21183662078059307
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.991152405453249, tolerance: 0.20260153622576405
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 68.8101153331304, tolerance: 0.21398472239780741
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.4584052386628628, tolerance: 0.21398472239780741
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.46057370272887965, tolerance: 0.20652709227686925
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.8246535440075604, tolerance: 2.139847223978074
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergen

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6517146931373645, tolerance: 0.21398472239780741
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 792.6452328990152, tolerance: 203.8066273711174
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 824.6823939169301, tolerance: 211.83662078059308
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 801.5212773805162, tolerance: 21.39847223978074
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 811.1580726229844, tolerance: 20.652709227686923
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 806.731625558684, tolerance: 2.038066273711174
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarnin

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 923.7072518351277, tolerance: 0.21183662078059307
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 890.4057855055984, tolerance: 0.20260153622576405
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 928.0836407513475, tolerance: 0.21398472239780741
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1029.2556670422803, tolerance: 20.652709227686923
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1015.8101661170265, tolerance: 2.038066273711174
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1055.9568646287241, tolerance: 2.1183662078059307
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceW

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'tol': [0.1, 0.01, 0.001, 0.0001]},
             verbose=False)

In [60]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [61]:
EN_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [62]:
EN.score(train_dataset_n,train_cible_n)

0.18272805940196757

In [ ]:
#Les meilleurs hyperparamètres.

In [63]:
EN.best_params_

{'alpha': 1, 'l1_ratio': 0.0, 'tol': 0.1}

In [ ]:
#D.3.2 Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [64]:
EN_best=ElasticNet(alpha=1, l1_ratio=0.0 , tol=0.1)

In [65]:
EN_best.fit(train_dataset_n,train_cible_n)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1132.2651817816018, tolerance: 259.70000000000016
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=1, l1_ratio=0.0, tol=0.1)

In [66]:
pd.DataFrame(EN_best.coef_, train_dataset_n.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.077630
PropertyGFATotal,0.178084
ZipCode_New,-0.043094
Campus,0.050211
Multifamily_HR,-0.035619
Multifamily_LR,0.020884
Multifamily_MR,-0.004425
NonResidential,0.006174
BALLARD,-0.015569
CENTRAL,-0.004909


In [ ]:
#D.3.3 Prédiction du meilleur modèle:

In [67]:
EN_best_prediction=EN_best.predict(test_dataset_n)
EN_best_prediction

array([ 4.37764441e-02,  1.40528654e-01,  3.31228464e-02,  8.77651754e-02,
        2.84562021e-01,  2.75555476e-01,  9.01409680e-02,  7.88845708e-01,
        2.75979887e-01,  3.09407774e-01,  6.15467465e-02, -2.73240027e-02,
        2.15046627e-01,  5.61965411e-01,  1.88114571e-02,  2.10073114e-01,
       -1.25808642e-02,  2.07041923e-01, -3.17883180e-01, -9.06517608e-02,
        1.57405766e-01, -4.75692506e-02, -7.42478692e-02,  5.60026225e-01,
        7.88403285e-01,  8.24797998e-02,  9.77228068e-01,  2.91974469e-01,
       -6.55066424e-02,  1.41628201e-01,  5.32331551e-02,  2.06746789e-01,
        6.36961257e-01,  2.44211850e-02,  7.53539971e-03,  3.69009602e-01,
       -2.70069194e-02,  1.87999231e-01,  8.68223278e-01,  4.25989483e-01,
       -1.35264124e-01,  1.59791913e-01, -9.42813486e-02,  8.17171337e-02,
        9.10100629e-02,  6.33554276e-01,  7.26002063e-02,  2.47419502e-01,
        3.28531860e-01,  1.22044561e+00,  6.73792671e-01,  5.73451508e-01,
        6.40225167e-01, -

In [ ]:
#D.3.4 Calcul de différentes métriques suite à la prédiction du meilleur modèle.

#Calcul du R^2:

In [68]:
R2_EN_best=r2_score(test_cible_n,EN_best_prediction)
R2_EN_best

0.21283653897300425

In [ ]:
#Calcul du MSE:

In [69]:
MSE_EN_best=mean_squared_error(test_cible_n,EN_best_prediction)

In [ ]:
#Calcul du RMSE:

In [70]:
RMSE_EN_best=math.sqrt(MSE_EN_best)

In [ ]:
#Calcul du MAE:

In [71]:
 MAE_EN_best=mean_absolute_error(test_cible_n,EN_best_prediction)

In [ ]:
#D.3.4 Résultats.

In [72]:
results_EN= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_EN= results_EN.append({'Modèle':'ElasticNet','R^2':R2_EN_best,'MSE':MSE_EN_best,'RMSE':RMSE_EN_best,'MAE':MAE_EN_best,'Temps de calcul (s)':EN_time.seconds},ignore_index=True)
results_EN

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,ElasticNet,0.212837,0.821893,0.906583,0.705609,14


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche que celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor et de la régression linéaire (ie. plus petite)

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor et proche à la régression linéaire.

#-----------------------------------------------------------------------



#D.4 RandomForestClassifier (Méthode ensembliste de régression non linéaire):

In [73]:
RF=RandomForestRegressor()

In [74]:
RF.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
#Le modèle random forest est:
#un méta estimateur qui adapte un certain nombre de classificateurs d'arbre de décision sur divers sous-échantillons de 
#l'ensemble de données et utilise la moyenne pour améliorer la précision prédictive et contrôler le sur ajustement.

#Les hyperparamètres intéressant à être modifiés sont:

# -n_estimators:= Le nombre d'arbres.

# -min_samples_leaf:= Le nombre minimum d'échantillons aux feuille de l'arbre.

# -max_features:= Le nombre de variables (ie. features) à considérer lors de la recherche de la meilleure répartition.

# -max_depth:= il module la profondeur pour éviter que le modèle sur apprenne.



#D.4.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [75]:
start_time = datetime.now()

In [ ]:
#On essaye plusieurs combinaisons pour ces hyperparamètres, afin de comparer les temps de camculs et les métriques.

In [76]:
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'max_depth' : [2,3,5,10,15,20,25,30,40,50 ,60], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt'], 'max_depth' : [2,3,5,10,15,20,30,40,50,60]}
parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10],'max_depth' : [2,3,5,10,20,40,60], 'max_features': ['sqrt']}


RF = GridSearchCV(estimator = RandomForestRegressor(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.4.1 Entraînement du modèle. 

In [77]:
RF.fit(train_dataset_n,train_cible_n)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 5, 10, 20, 40, 60],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5, 10],
                         'n_estimators': [10, 50, 100, 300, 500]},
             verbose=False)

In [78]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [79]:
RF_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [80]:
RF.score(train_dataset_n,train_cible_n)

0.9504733405514578

In [ ]:
#Les meilleurs hyperparamètres.

In [81]:
RF.best_params_

{'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 300}

In [ ]:
#D.4.2 Importance des variables ("features importances") pour le "meilleur" modèle (ie. celui avec les "meilleurs 
#hyperparamètres"):

In [82]:
RF_best=RandomForestRegressor(max_features='sqrt', min_samples_leaf=1, n_estimators=300, max_depth=40)

In [83]:
RF_best.fit(train_dataset_n,train_cible_n)

RandomForestRegressor(max_depth=40, max_features='sqrt', n_estimators=300)

In [84]:
pd.DataFrame(RF_best.feature_importances_, train_dataset_n.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.238204
PropertyGFATotal,0.554891
ZipCode_New,0.084185
Campus,0.012634
Multifamily_HR,0.010432
Multifamily_LR,0.005527
Multifamily_MR,0.000076
NonResidential,0.004293
BALLARD,0.003091
CENTRAL,0.005123


In [ ]:
#D.4.3 Prédiction.

In [85]:
RF_best_prediction=RF_best.predict(test_dataset_n)
RF_best_prediction

array([ 7.88110497e-01,  4.19228713e-01,  1.04248090e+00,  3.11888214e-01,
        1.26229804e+00,  9.07798977e-01,  6.94499680e-01,  2.12175298e+00,
        8.47539096e-01,  1.51398219e+00,  2.22348154e-01,  8.33712091e-01,
       -6.53446439e-01,  1.33178501e+00,  1.96671588e+00,  7.62625083e-01,
        7.05959134e-04,  9.38209972e-01, -6.00728064e-01,  9.32244126e-02,
       -6.06493876e-02,  5.61871100e-01,  6.01658563e-01,  2.03889463e+00,
        1.27700444e+00, -7.45425610e-01,  3.01568650e+00,  8.34431568e-01,
        9.40452588e-01, -3.32757441e-01, -7.13455383e-01,  1.15664372e+00,
        1.23351172e+00,  7.20566880e-01, -4.45220993e-01,  2.05781084e+00,
       -2.34599713e-01,  2.66291913e-01,  1.07113518e+00,  1.25989623e+00,
        1.98740276e-01,  2.13391238e-01,  1.97776539e-01,  8.56244078e-01,
        9.11952402e-01,  1.60054189e+00,  8.12236512e-02,  9.87290450e-01,
        9.75084642e-01,  1.54332798e+00,  1.02105893e+00,  1.68273565e+00,
        1.09886596e+00, -

In [ ]:
#D.4.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [86]:
R2_RF_best=r2_score(test_cible_n,RF_best_prediction)
R2_RF_best

0.7562894131900729

In [ ]:
#Calcul du MSE:

In [87]:
MSE_RF_best=mean_squared_error(test_cible_n,RF_best_prediction)

In [ ]:
#Calcul du RMSE:

In [88]:
RMSE_RF_best=math.sqrt(MSE_RF_best)

In [ ]:
#Calcul du MAE:

In [89]:
 MAE_RF_best=mean_absolute_error(test_cible_n,RF_best_prediction)

In [ ]:
#D.4.4 Résultats.

In [90]:
results_RF= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_RF= results_RF.append({'Modèle':'Random Forest','R^2':R2_RF_best,'MSE':MSE_RF_best,'RMSE':RMSE_RF_best,'MAE':MAE_RF_best,'Temps de calcul (s)':RF_time.seconds},ignore_index=True)
results_RF

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Random Forest,0.756289,0.254463,0.504443,0.346645,337


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche de cells du dummy regressor, la régression linéaire et le 
#  elasticNet.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor, la régression linéaire et le elasticNet.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor, de la régression linéaire et du elasticNet 
#  (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire et le elasticNet.

#-----------------------------------------------------------------------



#D.5 SVR (Méthode non linéaire):
    # "Support Vector Regression". Cette méthode cherche à minimiser les "pénalités" (L2 en particulier).
    #SVR nous donne la flexibilité de définir la quantité d'erreur acceptable dans notre modèle et trouvera une ligne 
    #appropriée (ou un hyperplan dans des dimensions plus élevées) pour s'adapter aux données.

In [91]:
SVR_1=SVR()

In [92]:
SVR_1.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
# Les hyperparamètres intéressant à être modifiés sont:

# -gamma:= Kernel coefficient. Il définit l'étendue de l'influence d'un seul exemple d'entraînement, les valeurs faibles 
#          signifiant "loin" et les valeurs élevées signifiant "proche". Les paramètres gamma peuvent être vus comme 
#          l'inverse du rayon d'influence des échantillons sélectionnés par le modèle comme vecteurs supports.

# -epsilon:= C'est le coefficient associé à la pénalité dans la fonction des pertes (loss fonction).

# -C:= Coefficient de régularisation. Il calibre le modèle et doit être positif pour éviter des over et under fit.
#      Le paramètre compromet la classification correcte des exemples d'apprentissage contre la maximisation de la marge de 
#      la fonction de décision. Pour des valeurs plus grandes de C, une marge plus petite sera acceptée si la fonction de 
#      décision réussit mieux à classer correctement tous les points d'apprentissage. Un C inférieur encouragera une plus 
#      grande marge, donc une fonction de décision plus simple, au détriment de la précision de l'apprentissage.




#D.5.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [93]:
start_time = datetime.now()

In [94]:
parameters = {'gamma' : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],'epsilon' : [0.001, 0.01, 0.1, 1], 'C' : [0.001, 0.01, 0.1, 1, 10]}

SVR_g= GridSearchCV(estimator=SVR_1, param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.5.1Entraînement du modèle. 

In [95]:
SVR_g.fit(train_dataset_n,train_cible_n)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.001, 0.01, 0.1, 1],
                         'gamma': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001,
                                   0.01, 0.1]},
             verbose=False)

In [96]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [97]:
SVR_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [98]:
SVR_g.score(train_dataset_n,train_cible_n)

0.4886262947819954

In [ ]:
#Les meilleurs hyperparamètres.

In [99]:
SVR_g.best_params_

{'C': 10, 'epsilon': 0.1, 'gamma': 0.1}

In [ ]:
#D.5.2 Développement du meilleur modèle:

In [100]:
SVR_best=SVR(C=1.0, epsilon=0.1, gamma=0.1)

In [101]:
SVR_best.fit(train_dataset_n,train_cible_n)

SVR(gamma=0.1)

In [ ]:
#D.5.3 Prédiction.

In [102]:
SVR_best_prediction=SVR_best.predict(test_dataset_n)
SVR_best_prediction

array([ 1.15516410e-01,  2.22296822e-01,  3.61465958e-01,  6.59955458e-02,
        8.53624681e-01,  2.72796915e-01, -1.18080367e-01,  1.35861114e+00,
        8.98328899e-01,  9.92181646e-01, -7.18366478e-01,  5.38363178e-01,
        7.01297112e-02,  1.21647868e+00,  6.25527075e-01,  1.49176247e-01,
       -3.10407156e-01,  2.99791122e-01, -3.06510205e-01, -6.68658750e-02,
       -1.24374578e-01,  5.97796211e-02,  7.13971745e-01,  2.08416173e+00,
        1.09763193e+00, -2.83015284e-01,  2.31916892e+00,  1.67975178e-01,
        1.60056288e-01, -9.76231176e-01,  2.10449010e-02,  9.50677147e-01,
        1.37639532e+00, -5.86461974e-02, -9.83520187e-02,  4.93813080e-01,
       -1.95799769e-01,  5.55800474e-01,  1.22922645e+00,  1.46878561e+00,
       -9.16366333e-02,  4.04204208e-01, -1.69987511e-01,  4.42614366e-01,
        5.26358043e-01,  1.20162929e+00,  3.87383955e-01,  2.62461738e-01,
        5.58406734e-01,  1.67436827e+00,  6.69742344e-01,  2.02173218e+00,
        2.01836208e+00, -

In [ ]:
#D.5.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [103]:
R2_SVR_best=r2_score(test_cible_n,SVR_best_prediction)
R2_SVR_best

0.39683418594587794

In [ ]:
#Calcul du MSE:

In [104]:
MSE_SVR_best=mean_squared_error(test_cible_n,SVR_best_prediction)

In [ ]:
#Calcul du RMSE:

In [105]:
RMSE_SVR_best=math.sqrt(MSE_SVR_best)

In [ ]:
#Calcul du MAE:

In [106]:
 MAE_SVR_best=mean_absolute_error(test_cible_n,SVR_best_prediction)

In [ ]:
#D.5.4 Résultats.

In [107]:
results_SVR= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_SVR= results_SVR.append({'Modèle':'SVR','R^2':R2_SVR_best,'MSE':MSE_SVR_best,'RMSE':RMSE_SVR_best,'MAE':MAE_SVR_best ,'Temps de calcul (s)':SVR_time.seconds},ignore_index=True)
results_SVR

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,SVR,0.396834,0.629777,0.793585,0.618638,457


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1 et de celle du random forest, mais plus proche que celles du dummy regressor, la
#  régression linéaire et le elasticNet.
# -la valeur du RMSE est supérieure à 0.5 et à de celle du random forest, mais plus proche que le dummy regressor, la
#  régression linéaire et le elasticNet.
# -La valeur du MAE est moins bonne celle du random forest, mais meilleure que celles du modèle naïf de dummy regressor, et
#  comparable à celle de la régression linéaire et du elasticNet (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire, le elasticNet et le random forest.


#---------------------------------------------------------------------------------------------------------------------------


#D.6 Tableau résumé:


In [108]:
results_total=np.concatenate((results_DM,results_LR,results_EN,results_RF),axis=1) 
results_total

array([['Dummy Regressor', -0.004425104178547912, 1.0487400216340284,
        1.0240800855568026, 0.792805190694773, 1, 'Linear regression',
        0.2892867907889912, 0.7420696508906133, 0.8614346469063183,
        0.6827409989752605, 5, 'ElasticNet', 0.21283653897300425,
        0.8218928636019245, 0.9065830704364187, 0.7056090529693929, 14,
        'Random Forest', 0.7562894131900729, 0.254463020707267,
        0.5044432779879885, 0.3466446392283684, 337]], dtype=object)

In [109]:
results_total= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_total= results_total.append({'Modèle':'Dummy Regressor','R^2':R2_DM,'MSE':MSE_DM,'RMSE':RMSE_DM,'MAE':MAE_DM ,'Temps de calcul (s)':DM_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'Linear Regression','R^2':R2_LR,'MSE':MSE_LR,'RMSE':RMSE_LR,'MAE':MAE_LR ,'Temps de calcul (s)':LR_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'ElasticNet','R^2':R2_EN_best,'MSE':MSE_EN_best,'RMSE':RMSE_EN_best,'MAE':MAE_EN_best ,'Temps de calcul (s)':EN_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'Random Forest','R^2':R2_RF_best,'MSE':MSE_RF_best,'RMSE':RMSE_RF_best,'MAE':MAE_RF_best ,'Temps de calcul (s)':RF_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'SVR','R^2':R2_SVR_best,'MSE':MSE_SVR_best,'RMSE':RMSE_SVR_best,'MAE':MAE_SVR_best ,'Temps de calcul (s)':SVR_time.seconds},ignore_index=True)
results_total

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.004425,1.048740,1.024080,0.792805,1
1,Linear Regression,0.289287,0.742070,0.861435,0.682741,5
2,ElasticNet,0.212837,0.821893,0.906583,0.705609,14
3,Random Forest,0.756289,0.254463,0.504443,0.346645,337
4,SVR,0.396834,0.629777,0.793585,0.618638,457


In [ ]:
# On regarde que le meilleur modèle est de loin le Random Forest avec:
# -un temps de calcul (ie. temps d'entraînement) d'un peu moins de 7.5 min, 
# -un R^2 qui reste plus petit que 1 mais qui déjà dépasse les 0.5 (On rappelle que à partir de 0.6 on considère que le R^2
#  a une valeur robuste),
# -la MAE la plus petite de tous les modèles,
# -une RMSE bien plus petite que pour les autres modèles, même si elle reste supérieure à 0.5 (pour rappel, des valeurs de 
#  RMSE comprises entre 0.2 et 0.5 montrent que le modèle peut prédire relativement les données avec précision).

#Notez aussi qu'on perçoit que le R^2 n'est pas forcément le meilleure métrique. En fait on sait bien qu'un "mauvais" R^2 ne
#signifie pas que le modèle est forcément mauvais ou inapproprié, on peut avoir des modèles avec mauvais R^2 mais qui 
#restent performants et bon. C'est pour cela que l'on préfère considérer aussi le RMSE et le MAE pour valider nos modèles.


#---------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------


#Faisons maintemant la même chose mais en considérant parmi nos varibles descriptives (ie.features) la variable 
#ENERGYSTARScore

In [ ]:
#---

In [ ]:
#---

In [ ]:
#A. IMPORTER LES DONNÉES:

In [110]:
df_E=pd.read_csv('Data_clean_E.csv', sep=',', low_memory=False)

In [111]:
df_E.count()

OSEBuildingID                    2172
TaxParcelIdentificationNumber    2172
PropertyName                     2172
SourceEUI_kBtu_sf_log            2172
TotalGHGEmissions_log            2172
BuildingType                     2172
YearBuilt                        2172
PropertyGFATotal                 2172
Neighborhood                     2172
ZipCode_New                      2172
ENERGYSTARScore                  2172
dtype: int64

In [112]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)

In [ ]:
#B.1.1 'BuildingType':

In [113]:
encoder_BuildingType_E = pd.DataFrame(encoder.fit_transform(df_E[['BuildingType']]))
encoder_BuildingType_E

,0,1,2,3
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
2167,0.0,0.0,1.0,0.0
2168,0.0,1.0,0.0,0.0
2169,0.0,1.0,0.0,0.0
2170,0.0,1.0,0.0,0.0


In [114]:
encoder_BuildingType_E.rename(columns={0:'Campus',1:'Multifamily_HR',2:'Multifamily_LR',3:'Multifamily_MR', 4:'NonResidential', 5:'NonResidential_COS', 6:'NonResidential_WA', 7:'SPS-District_K-12'},inplace = True)
encoder_BuildingType_E

,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
2167,0.0,0.0,1.0,0.0
2168,0.0,1.0,0.0,0.0
2169,0.0,1.0,0.0,0.0
2170,0.0,1.0,0.0,0.0


In [ ]:
#B.1.2 'Neighborhood':

In [115]:
encoder_Neighborhood_E = pd.DataFrame(encoder.fit_transform(df_E[['Neighborhood']]))
encoder_Neighborhood_E

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2169,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2170,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
encoder_Neighborhood_E.rename(columns={0:'BALLARD',1:'CENTRAL',2:'DELRIDGE',3:'DOWNTOWN', 4:'EAST', 5:'GREATER_DUWAMISH', 6:'LAKE_UNION', 7:'MAGNOLIA_QUEEN_ANNE', 8:'NORTH', 9:'NORTHEAST', 10:'NORTHWEST', 11:'SOUTHEAST', 12:'SOUTHWEST' },inplace = True)
encoder_Neighborhood_E

,BALLARD,CENTRAL,DELRIDGE,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2169,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2170,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#B.2 On fusionne les colonnes résultant du OneHotEncoder dans le tableau initial.

In [117]:
f1_df_E = df_E.join(encoder_BuildingType_E)

In [118]:
final_df_E= f1_df_E.join(encoder_Neighborhood_E)
final_df_E.count()

OSEBuildingID                    2172
TaxParcelIdentificationNumber    2172
PropertyName                     2172
SourceEUI_kBtu_sf_log            2172
TotalGHGEmissions_log            2172
BuildingType                     2172
YearBuilt                        2172
PropertyGFATotal                 2172
Neighborhood                     2172
ZipCode_New                      2172
ENERGYSTARScore                  2172
Campus                           2172
Multifamily_HR                   2172
Multifamily_LR                   2172
Multifamily_MR                   2172
BALLARD                          2172
CENTRAL                          2172
DELRIDGE                         2172
DOWNTOWN                         2172
EAST                             2172
GREATER_DUWAMISH                 2172
LAKE_UNION                       2172
MAGNOLIA_QUEEN_ANNE              2172
NORTH                            2172
NORTHEAST                        2172
NORTHWEST                        2172
SOUTHEAST   

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#C.Le dataset.

#C.1 Élimination des variables non nécessaires.

#On élimine les colonnes des variables catégorielles initiales et celles d'identification des bâtiments:
#'OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'.
#On reste aussi uniquement avec la variable cible qui nous intéresse: 'TotalGHGEmissions_log'.

In [119]:
final_df_E.drop({'SourceEUI_kBtu_sf_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#C.2 Le dataset.

In [120]:
dataset_E=final_df_E
dataset_E.head(5)

,TotalGHGEmissions_log,YearBuilt,PropertyGFATotal,ZipCode_New,ENERGYSTARScore,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,5.523179,1927,88434,98101,65.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.577879,1996,103566,98101,51.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7.631664,1969,961990,98101,18.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.231858,1980,119890,98121,67.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.344533,1926,83008,98101,25.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#C.3 Training et Test sets.

#On divise les données en 2 sets: un "training set" et un "test set".

In [121]:
train_dataset_E = dataset_E.sample(frac=0.8,random_state=0)
test_dataset_E = dataset_E.drop(train_dataset_E.index)
train_dataset_E

,TotalGHGEmissions_log,YearBuilt,PropertyGFATotal,ZipCode_New,ENERGYSTARScore,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
194,4.740575,1916,350310,98121,89.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303,5.902360,1991,323916,98108,81.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1205,5.421375,2006,269297,98115,80.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1695,2.430098,1972,48112,98133,73.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1981,4.186468,1962,51410,98116,87.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,8.753357,1920,374466,98101,74.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1843,3.231200,1984,57110,98119,69.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1932,5.173491,1970,44000,98108,15.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1556,3.234355,1965,27800,98108,13.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
train_dataset_E.sum()

TotalGHGEmissions_log    6.969340e+03
YearBuilt                3.413324e+06
PropertyGFATotal         2.254345e+08
ZipCode_New              1.705263e+08
ENERGYSTARScore          1.122280e+05
Campus                   7.000000e+00
Multifamily_HR           1.580000e+03
Multifamily_LR           1.400000e+01
Multifamily_MR           1.370000e+02
BALLARD                  7.100000e+01
CENTRAL                  5.200000e+01
DELRIDGE                 4.600000e+01
DOWNTOWN                 4.070000e+02
EAST                     1.140000e+02
GREATER_DUWAMISH         3.400000e+02
LAKE_UNION               1.650000e+02
MAGNOLIA_QUEEN_ANNE      1.510000e+02
NORTH                    6.800000e+01
NORTHEAST                1.330000e+02
NORTHWEST                9.100000e+01
SOUTHEAST                5.900000e+01
SOUTHWEST                4.100000e+01
dtype: float64

In [ ]:
#C.4 Quelques statistiques.

In [123]:
train_stats_E = train_dataset_E.describe()
train_stats_E = train_stats_E.transpose()
train_stats_E

,count,mean,std,min,25%,50%,75%,max
TotalGHGEmissions_log,1738.0,4.009977,1.395159,-1.609438,3.075121,3.947293,4.940678,9.733410e+00
YearBuilt,1738.0,1963.937860,31.841317,1900.000000,1940.000000,1968.000000,1989.000000,2.015000e+03
PropertyGFATotal,1738.0,129709.138665,212468.864690,16300.000000,32975.250000,55279.500000,124472.000000,1.952220e+06
ZipCode_New,1738.0,98116.404488,17.296471,98020.000000,98104.000000,98109.000000,98122.000000,9.819900e+04
ENERGYSTARScore,1738.0,64.573072,28.440620,1.000000,48.000000,72.000000,89.000000,1.000000e+02
Campus,1738.0,0.004028,0.063354,0.000000,0.000000,0.000000,0.000000,1.000000e+00
Multifamily_HR,1738.0,0.909091,0.287563,0.000000,1.000000,1.000000,1.000000,1.000000e+00
Multifamily_LR,1738.0,0.008055,0.089414,0.000000,0.000000,0.000000,0.000000,1.000000e+00
Multifamily_MR,1738.0,0.078826,0.269545,0.000000,0.000000,0.000000,0.000000,1.000000e+00
BALLARD,1738.0,0.040852,0.198003,0.000000,0.000000,0.000000,0.000000,1.000000e+00


In [ ]:
#C.5 Normalisation des données.

#Il est recommandé de normaliser les variables qui ont différentes échelles (cf. statistiques en haut). En fait, même si 
#le modèle peut converger sans normalisation ça rend l'entraînement plus difficile et le résultat dépendant des unités 
#utilisées pour les variables.

#Notez que nous avons générer les statistiques précédentes uniquement sur le "set training", ces statistiques sont également
#utilisées pour normaliser l'ensemble de données de test, ceci l'ensemble de données de test doivent être projetées dans la 
#même distribution que celle sur laquelle le training fut fait.

#On a décidé de normaliser aussi la variable cible. En fait même si ceci n'est pas vraiment nécessaire, c'est juste que des 
#fois si on ne normalise pas la variable cible il peut y avoir des bias et on a préféré d'éviter ceci depuis le début.

In [124]:
def norm(x):
  return (x - train_stats_E['mean']) / train_stats_E['std']
train_dataset_En = norm(train_dataset_E)
test_dataset_En = norm(test_dataset_E)

In [125]:
train_stats_En = train_dataset_En.describe()
train_stats_En = train_stats_En.transpose()
train_stats_En

,count,mean,std,min,25%,50%,75%,max
TotalGHGEmissions_log,1738.0,-4.968535e-16,1.0,-4.027795,-0.670071,-0.044929,0.667093,4.102351
YearBuilt,1738.0,2.292246e-15,1.0,-2.008016,-0.751786,0.127575,0.787095,1.603644
PropertyGFATotal,1738.0,1.065987e-18,1.0,-0.533768,-0.455285,-0.350308,-0.024649,8.577779
ZipCode_New,1738.0,2.672597e-13,1.0,-5.573651,-0.717169,-0.428092,0.323506,4.775281
ENERGYSTARScore,1738.0,-2.065698e-16,1.0,-2.235291,-0.582725,0.261138,0.858875,1.245645
Campus,1738.0,3.230379e-16,1.0,-0.063573,-0.063573,-0.063573,-0.063573,15.720796
Multifamily_HR,1738.0,-1.583569e-16,1.0,-3.161368,0.316137,0.316137,0.316137,0.316137
Multifamily_LR,1738.0,-1.182582e-15,1.0,-0.090089,-0.090089,-0.090089,-0.090089,11.093782
Multifamily_MR,1738.0,8.240436e-17,1.0,-0.292442,-0.292442,-0.292442,-0.292442,3.417515
BALLARD,1738.0,-1.244689e-16,1.0,-0.206318,-0.206318,-0.206318,-0.206318,4.844106


In [ ]:
#C.6 Séparez la variable cible des variables prédictives. 

#La variable cible est la valeur que l'on entraînera le modèle à prédire.

In [126]:
train_cible_En = train_dataset_En.pop('TotalGHGEmissions_log')
test_cible_En = test_dataset_En.pop('TotalGHGEmissions_log')

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#D. Les Modèles.
#Construction et entraînement des modèles.
    #Notez que la fonction ".fit", c'est celle qui est chargée d'entraîner notre modèle et que la fonction ".predict", c'est
    #celle qui est chargée de prédire la variable cible à partir du dataset test.
    
    
#------

#D.1 DummyRegressor: 
    #On prend la médiane car elle est statistiquement préférable à la moyenne.
    #Sans optimisation des hyperparamètres (pour le DummyRegressor il n'y a pas vraiment des paramètres à optimiser).

In [127]:
DM_E= DummyRegressor(strategy='median')

In [ ]:
#D.1.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [128]:
start_time_E = datetime.now()

In [129]:
DM_E.fit(train_dataset_En,train_cible_En)

DummyRegressor(strategy='median')

In [130]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [131]:
DM_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [132]:
DM_E.score(train_dataset_En,train_cible_En)

-0.0020198180357982043

In [ ]:
#Notez que la valeur absolue du R^2 (qui est donné par le "score") est loin de 1, ce qui montre la faiblesse de ce 
#modèle.

#D.1.2 Prédiction.

In [133]:
DM_prediction_E=DM_E.predict(test_dataset_En)
DM_prediction_E

array([-0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04492945,
       -0.04492945, -0.04492945, -0.04492945, -0.04492945, -0.04

In [ ]:
#D.1.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [134]:
R2_DM_E=r2_score(test_cible_En,DM_prediction_E)
R2_DM_E

-0.0018506958079205216

In [ ]:
#Calcul du MSE:

In [135]:
MSE_DM_E=mean_squared_error(test_cible_En,DM_prediction_E)

In [ ]:
#Calcul du RMSE:

In [136]:
RMSE_DM_E=math.sqrt(MSE_DM_E)

In [ ]:
#Calcul du MAE:

In [137]:
 MAE_DM_E=mean_absolute_error(test_cible_En,DM_prediction_E)

In [ ]:
#D.1.5 Résultats.

In [138]:
results_DM_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_DM_E= results_DM_E.append({'Modèle':'Dummy Regressor','R^2':R2_DM_E,'MSE':MSE_DM_E,'RMSE':RMSE_DM_E,'MAE':MAE_DM_E ,'Temps de calcul (s)':DM_time_E.seconds},ignore_index=True)
results_DM_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.001851,1.018274,1.009096,0.778984,1


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1.
# -la valeur du RMSE est supérieure à 0.5 (des valeurs de RMSE comprises entre 0.2 et 0.5 montrent que le modèle peut 
#  prédire relativement les données avec précision) et
# -La valeur du MAE de modèle naïf de Random forest servira de base, on rappelle qu'il n'y a pas de valeur correcte pour la 
#  MSE. En termes simples, plus la valeur est faible, mieux c'est et 0 signifie que le modèle est parfait. On se base dans
#  la MEA d'un modèle naïf (comme le dummy regressor) pour comparer avec la MEA de modèles plus performants.

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle.

# Notez qu'en considérant le ENERGYSTARScore on n'améliore pas les valeurs des métriques pour le dummy regressor.
#-----------------------------------------------------------------------

#D.2 LinearRegression.
    #Sans optimisation des hyperparamètres (pour la régression linéaire il n'y a pas vraiment des paramètres à optimiser).

In [139]:
LR_E=LinearRegression()

In [140]:
LR_E.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': False,
 'positive': False}

In [ ]:
#D.2.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [141]:
start_time_E = datetime.now()

In [142]:
LR_E.fit(train_dataset_En,train_cible_En)

LinearRegression()

In [143]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [144]:
LR_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [145]:
LR_E.score(train_dataset_En,train_cible_En)

0.3589502026863738

In [ ]:
#D.2.2 Prédiction.

In [146]:
LR_prediction_E=LR_E.predict(test_dataset_En)
LR_prediction_E

array([ 9.09608390e-01, -1.50206063e-01,  3.63465812e-01,  2.29632714e+00,
       -3.19883798e-01, -1.38731454e-01, -4.49766610e-01,  6.47401359e-01,
        2.29096915e-01,  1.49842765e-01,  6.11268546e-01,  5.09217765e-01,
       -6.45127038e-02,  5.18159415e-01, -2.54209969e-01,  4.33495249e-03,
        6.78163077e-01, -1.96348641e-01, -3.20860360e-01,  1.07464745e+00,
        6.95497062e-01,  4.69178702e-01,  5.56825187e-01,  5.12635734e-01,
        5.97596671e-01, -1.50938485e-01,  1.38910058e+00,  7.66005775e-02,
        6.58583900e-02,  4.36952140e-01, -4.94239126e-03,  1.19476464e+00,
        6.19521400e-02,  2.89491202e-01, -2.38340829e-01,  1.01947167e+00,
       -5.47860009e-04,  4.16884681e-02,  5.64393546e-01, -5.86533288e-02,
        4.29091712e-02,  1.79139640e-01, -1.56845788e-02,  3.70497962e-02,
        6.28807749e-03,  1.62933495e+00,  3.33924796e-01, -8.84384850e-02,
       -3.60167001e-01, -2.73252938e-01, -2.72032235e-01, -1.45079110e-01,
        9.61318275e-02,  

In [ ]:
#D.2.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [147]:
R2_LR_E=r2_score(test_cible_En,LR_prediction_E)
R2_LR_E

0.39208355548895213

In [ ]:
#Calcul du MSE:

In [148]:
MSE_LR_E=mean_squared_error(test_cible_En,LR_prediction_E)

In [ ]:
#Calcul du RMSE:

In [149]:
RMSE_LR_E=math.sqrt(MSE_LR_E)

In [ ]:
#Calcul du MAE:

In [150]:
 MAE_LR_E=mean_absolute_error(test_cible_En,LR_prediction_E)

In [ ]:
#D.2.5 Résultats.

In [151]:
results_LR_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_LR_E= results_LR_E.append({'Modèle':'Linear regression','R^2':R2_LR_E,'MSE':MSE_LR_E,'RMSE':RMSE_LR_E,'MAE':MAE_LR_E,'Temps de calcul (s)':LR_time_E.seconds},ignore_index=True)
results_LR_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Linear regression,0.392084,0.617882,0.786055,0.612551,1


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche que celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celle du modèle naïf de dummy regressor(ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor.

# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques pour la linear regression.
# mais vaut-il vraiment le coup pour cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?

#D.2.6 Coefficients obtenus par le modèle:

In [152]:
pd.DataFrame(LR_E.coef_, train_dataset_En.columns, columns = ['Coeff'])

,Coeff
YearBuilt,1.271434e-02
PropertyGFATotal,5.252481e-01
ZipCode_New,-2.122185e-02
ENERGYSTARScore,-2.447793e-01
Campus,-7.360617e+11
Multifamily_HR,-3.340979e+12
Multifamily_LR,-1.038842e+12
Multifamily_MR,-3.131646e+12
BALLARD,-7.927070e+11
CENTRAL,-6.822541e+11


In [ ]:
#-----------------------------------------------------------------------

#D.3 ElasticNet (méthode de régression linéaire).

    #C'est une extension de la régression linéaire, avec un paramètre de pénalité supplémentaire qui vise à minimiser la 
    #complexité et/ou à réduire le nombre de variables (features) utilisées dans le modèle final, ce qui rend intéressant 
    #le fait de le tester. Quoi qu'il en soit, 'objectif -comme pour de nombreux modèles- est de réduire l'erreur de 
    #l'ensemble de test.
    #ElasticNet est un mélange de Lasso (qui inclu la pénalité L1) et Ridge Regression (qui inclue la pénalité L2).

In [153]:
EN_E=ElasticNet()

In [154]:
EN_E.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [ ]:
#Le modèle ElasticNet est:
#(1 / (2 * n_samples) )* ||y - Xw||^2_2 + alpha * l1_ratio * ||w||_1 + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

#Les hyperparamètres intéressant à être modifiés sont:

# -alpha:= attribue le poids accordé à chacune des pénalités L1 et L2.

# -l1_ratio:= C'est un paramètre de pénalité de mélange. Pour l1_ratio = 0 la pénalité est une pénalité L2 (Ridge). 
#             Pour l1_ratio = 1 c'est une pénalité L1 (Lasso). Pour 0 < l1_ratio < 1, la pénalité est une combinaison de L1 
#             et L2.

# -tol:= La tolérance d'optimisation, ie. quand le modèle arrête-t-il le processus d'optimisation.



#D.3.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [155]:
start_time_E = datetime.now()

In [156]:
parameters = {'tol' : [0.1,0.01,0.001,0.0001], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': np.arange(0.0, 1.0, 0.1)} #0: Ridge; 1:Lasso

EN_E = GridSearchCV(estimator = ElasticNet(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.3.1 Entraînement du modèle.

In [157]:
EN_E.fit(train_dataset_En,train_cible_En)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 453.53853431005535, tolerance: 142.32869851152077
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 440.05066284582875, tolerance: 139.89446354983252
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 450.9844736732723, tolerance: 140.50275912288166
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceW

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.936529352155276, tolerance: 1.4232869851152077
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.704510355236266, tolerance: 1.3989446354983253
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.502282157548279, tolerance: 1.4050275912288166
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.907169986489976, tolerance: 1.4232869851152077
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.907169986489976, tolerance: 0.14232869851152077
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0334785256097803, tolerance: 0.13989446354983254
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 440.30600804929986, tolerance: 139.89446354983252
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 451.20751582608483, tolerance: 140.50275912288166
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 443.47212184675175, tolerance: 137.72044669018788
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1996513723111093, tolerance: 0.13989446354983254
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.22283400411004095, tolerance: 0.14050275912288168
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.14199508629144475, tolerance: 0.13772044669018788
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conver

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 453.41220700674694, tolerance: 0.14050275912288168
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 445.6537101521015, tolerance: 0.13772044669018788
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 434.75037441331506, tolerance: 0.13425680578280152
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergen

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 568.3637982690736, tolerance: 13.989446354983253
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 569.7726693662944, tolerance: 14.050275912288166
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 560.2446269450037, tolerance: 13.772044669018788
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 673.7195109406008, tolerance: 0.13989446354983254
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 675.6281018877065, tolerance: 0.14050275912288168
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 662.8216714182364, tolerance: 0.13772044669018788
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'tol': [0.1, 0.01, 0.001, 0.0001]},
             verbose=False)

In [158]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [159]:
EN_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [160]:
EN_E.score(train_dataset_En,train_cible_En)

0.3584040281829505

In [ ]:
#Les meilleurs hyperparamètres.

In [161]:
EN_E.best_params_

{'alpha': 0.01, 'l1_ratio': 0.5, 'tol': 0.01}

In [ ]:
#D.3.2 Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [162]:
EN_best_E=ElasticNet(alpha=0.01, l1_ratio=0.30000000000000004 , tol=0.01)

In [163]:
EN_best_E.fit(train_dataset_En,train_cible_En)

ElasticNet(alpha=0.01, l1_ratio=0.30000000000000004, tol=0.01)

In [164]:
pd.DataFrame(EN_best_E.coef_, train_dataset_En.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.009434
PropertyGFATotal,0.519093
ZipCode_New,-0.019461
ENERGYSTARScore,-0.236432
Campus,0.041206
Multifamily_HR,-0.094717
Multifamily_LR,-0.023552
Multifamily_MR,0.018079
BALLARD,-0.007600
CENTRAL,-0.032543


In [ ]:
#D.3.3 Prédiction du meilleur modèle:

In [165]:
EN_best_prediction_E=EN_best_E.predict(test_dataset_En)
EN_best_prediction_E

array([ 8.83706494e-01, -1.50070960e-01,  3.37679200e-01,  2.26120682e+00,
       -3.17297647e-01, -1.22507450e-01, -4.21216909e-01,  6.16736545e-01,
        1.93623840e-01,  1.39636391e-01,  6.16236440e-01,  5.01303631e-01,
       -7.51530568e-02,  4.82144277e-01, -2.41862059e-01,  1.91173742e-02,
        6.69267806e-01, -1.79610901e-01, -3.26953301e-01,  1.07883688e+00,
        6.58090526e-01,  4.36429173e-01,  5.52993398e-01,  4.93012638e-01,
        5.79723740e-01, -1.52059381e-01,  1.37043234e+00,  7.60668822e-02,
        6.44017020e-02,  4.36126138e-01, -5.42432442e-03,  1.17729998e+00,
        6.60784925e-02,  3.05368609e-01, -2.29222581e-01,  1.00456589e+00,
       -6.83938379e-03,  3.13305732e-02,  5.54978054e-01, -5.89875984e-02,
        4.19281145e-02,  1.69860197e-01, -1.40102600e-02,  3.59357882e-02,
        5.43036821e-05,  1.61417450e+00,  3.36512563e-01, -7.48852244e-02,
       -3.42218516e-01, -2.60464643e-01, -2.55139191e-01, -1.32012367e-01,
        1.11342820e-01,  

In [ ]:
#D.3.4 Calcul de différentes métriques suite à la prédiction du meilleur modèle.

#Calcul du R^2:

In [166]:
R2_EN_best_E=r2_score(test_cible_En,EN_best_prediction_E)
R2_EN_best_E

0.39040802175076694

In [ ]:
#Calcul du MSE:

In [167]:
MSE_EN_best_E=mean_squared_error(test_cible_En,EN_best_prediction_E)

In [ ]:
#Calcul du RMSE:

In [168]:
RMSE_EN_best_E=math.sqrt(MSE_EN_best_E)

In [ ]:
#Calcul du MAE:

In [169]:
 MAE_EN_best_E=mean_absolute_error(test_cible_En,EN_best_prediction_E)

In [ ]:
#D.3.4 Résultats.

In [170]:
results_EN_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_EN_E= results_EN_E.append({'Modèle':'ElasticNet','R^2':R2_EN_best_E,'MSE':MSE_EN_best_E,'RMSE':RMSE_EN_best_E,'MAE':MAE_EN_best_E,'Temps de calcul (s)':EN_time_E.seconds},ignore_index=True)
results_EN_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,ElasticNet,0.390408,0.619585,0.787137,0.613251,25


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche que celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor et de la régression linéaire (ie. plus petite)

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor et proche à la régression linéaire.


# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques, mais vaut-il vraiment le coup pour
#cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?

#-----------------------------------------------------------------------



#D.4 RandomForestClassifier (Méthode ensembliste de régression non linéaire):

In [171]:
RF_E=RandomForestRegressor()

In [172]:
RF_E.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
#Le modèle random forest est:
#un méta estimateur qui adapte un certain nombre de classificateurs d'arbre de décision sur divers sous-échantillons de 
#l'ensemble de données et utilise la moyenne pour améliorer la précision prédictive et contrôler le sur ajustement.

#Les hyperparamètres intéressant à être modifiés sont:

# -n_estimators:= Le nombre d'arbres.

# -min_samples_leaf:= Le nombre minimum d'échantillons aux feuille de l'arbre.

# -max_features:= Le nombre de variables (ie. features) à considérer lors de la recherche de la meilleure répartition.

# -max_depth:= il module la profondeur pour éviter que le modèle sur apprenne.



#D.4.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [173]:
start_time_E = datetime.now()

In [ ]:
#On essaye plusieurs combinaisons pour ces hyperparamètres, afin de comparer les temps de camculs et les métriques.

In [174]:
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'max_depth' : [2,3,5,10,15,20,25,30,40,50 ,60], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt'], 'max_depth' : [2,3,5,10,15,20,30,40,50,60]}
parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10],'max_depth' : [2,3,5,10,20,40,60], 'max_features': ['sqrt']}


RF_E = GridSearchCV(estimator = RandomForestRegressor(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.4.1 Entraînement du modèle. 

In [175]:
RF_E.fit(train_dataset_En,train_cible_En)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 5, 10, 20, 40, 60],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5, 10],
                         'n_estimators': [10, 50, 100, 300, 500]},
             verbose=False)

In [176]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [177]:
RF_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [178]:
RF_E.score(train_dataset_En,train_cible_En)

0.9638522962057733

In [ ]:
#Les meilleurs hyperparamètres.

In [179]:
RF_E.best_params_

{'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 500}

In [ ]:
#D.4.2 Importance des variables ("features importances") pour le "meilleur" modèle (ie. celui avec les "meilleurs 
#hyperparamètres"):

In [180]:
RF_best_E=RandomForestRegressor(max_features='sqrt', min_samples_leaf=1, n_estimators=100, max_depth=60)

In [181]:
RF_best_E.fit(train_dataset_En,train_cible_En)

RandomForestRegressor(max_depth=60, max_features='sqrt')

In [182]:
pd.DataFrame(RF_best_E.feature_importances_, train_dataset_En.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.163848
PropertyGFATotal,0.462723
ZipCode_New,0.083804
ENERGYSTARScore,0.168012
Campus,0.003957
Multifamily_HR,0.007252
Multifamily_LR,0.001496
Multifamily_MR,0.005228
BALLARD,0.004551
CENTRAL,0.004643


In [ ]:
#D.4.3 Prédiction.

In [183]:
RF_best_prediction_E=RF_best_E.predict(test_dataset_En)
RF_best_prediction_E

array([ 1.64986511e+00, -5.38745649e-01,  1.10113332e+00,  2.22760670e+00,
        5.07076034e-01,  5.23727832e-01,  5.55996466e-02,  5.96419419e-01,
       -5.43095986e-01, -3.59206733e-01,  1.84724411e+00,  1.06093196e+00,
        4.82199787e-01,  1.11735553e+00,  6.04607400e-01,  7.83271120e-01,
        8.05452234e-01,  6.31904848e-01,  3.04771291e-01,  2.26570485e+00,
        8.97820058e-01,  6.70443994e-01,  1.34988388e+00,  1.43201217e+00,
        8.49960977e-01,  7.26468957e-02,  1.14965045e+00,  9.43459471e-01,
       -1.27916794e-01,  5.07125403e-01,  5.44591977e-01,  1.04612592e+00,
        7.40658477e-01,  1.03807006e+00,  3.76171185e-01,  1.33239276e+00,
        5.95214425e-01,  1.83428284e-01,  1.62963185e+00,  4.72320801e-01,
        5.32769522e-01,  2.08043490e-01,  6.79752767e-01,  3.09656775e-01,
        6.68374432e-02,  1.22291057e+00,  7.92750906e-01, -9.38287745e-02,
       -4.14256414e-01, -4.84818976e-01, -5.31993100e-01, -2.15409440e-01,
        3.13954423e-01,  

In [ ]:
#D.4.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [184]:
R2_RF_best_E=r2_score(test_cible_En,RF_best_prediction_E)
R2_RF_best_E

0.7523033570557086

In [ ]:
#Calcul du MSE:

In [185]:
MSE_RF_best_E=mean_squared_error(test_cible_En,RF_best_prediction_E)

In [ ]:
#Calcul du RMSE:

In [186]:
RMSE_RF_best_E=math.sqrt(MSE_RF_best_E)

In [ ]:
#Calcul du MAE:

In [187]:
 MAE_RF_best_E=mean_absolute_error(test_cible_En,RF_best_prediction_E)

In [ ]:
#D.4.4 Résultats.

In [188]:
results_RF_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_RF_E= results_RF_E.append({'Modèle':'Random Forest','R^2':R2_RF_best_E,'MSE':MSE_RF_best_E,'RMSE':RMSE_RF_best_E,'MAE':MAE_RF_best_E,'Temps de calcul (s)':RF_time_E.seconds},ignore_index=True)
results_RF_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Random Forest,0.752303,0.251757,0.501754,0.358595,411


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche de 0.6 et de celles du dummy regressor, la régression 
#  linéaire et le elasticNet.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor, la régression linéaire et le elasticNet.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor, de la régression linéaire et du elasticNet 
#  (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire et le elasticNet.


# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques, mais vaut-il vraiment le coup pour
# cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?



#-----------------------------------------------------------------------



#D.5 SVR (Méthode non linéaire):
    # "Support Vector Regression". Cette méthode cherche à minimiser les "pénalités" (L2 en particulier).
    #SVR nous donne la flexibilité de définir la quantité d'erreur acceptable dans notre modèle et trouvera une ligne 
    #appropriée (ou un hyperplan dans des dimensions plus élevées) pour s'adapter aux données.

In [189]:
SVR_E=SVR()

In [190]:
SVR_E.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
#Les hyperparamètres intéressant à être modifiés sont:

# -gamma:= Kernel coefficient. Il définit l'étendue de l'influence d'un seul exemple d'entraînement, les valeurs faibles 
#          signifiant "loin" et les valeurs élevées signifiant "proche". Les paramètres gamma peuvent être vus comme 
#          l'inverse du rayon d'influence des échantillons sélectionnés par le modèle comme vecteurs supports.

# -epsilon:= C'est le coefficient associé à la pénalité dans la fonction des pertes (loss fonction).

# -C:= Coefficient de régularisation. Il calibre le modèle et doit être positif pour éviter des over et under fit.
#      Le paramètre compromet la classification correcte des exemples d'apprentissage contre la maximisation de la marge de 
#      la fonction de décision. Pour des valeurs plus grandes de C, une marge plus petite sera acceptée si la fonction de 
#      décision réussit mieux à classer correctement tous les points d'apprentissage. Un C inférieur encouragera une plus 
#      grande marge, donc une fonction de décision plus simple, au détriment de la précision de l'apprentissage.



#D.5.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [191]:
start_time_E = datetime.now()

In [192]:
parameters = {'gamma' : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],'epsilon' : [0.001, 0.01, 0.1, 1], 'C' : [0.001, 0.01, 0.1, 1, 10]}

SVR_g_E= GridSearchCV(estimator=SVR_E, param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.5.1Entraînement du modèle. 

In [193]:
SVR_g_E.fit(train_dataset_En,train_cible_En)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.001, 0.01, 0.1, 1],
                         'gamma': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001,
                                   0.01, 0.1]},
             verbose=False)

In [194]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [195]:
SVR_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [196]:
SVR_g_E.score(train_dataset_En,train_cible_En)

0.6302201059643959

In [ ]:
#Les meilleurs hyperparamètres.

In [197]:
SVR_g_E.best_params_

{'C': 10, 'epsilon': 0.1, 'gamma': 0.1}

In [ ]:
#D.5.2 Développement du meilleur modèle:

In [198]:
SVR_best_E=SVR(C=10, epsilon=0.1, gamma=0.1)

In [199]:
SVR_best_E.fit(train_dataset_En,train_cible_En)

SVR(C=10, gamma=0.1)

In [ ]:
#D.5.3 Prédiction.

In [200]:
SVR_best_prediction_E=SVR_best_E.predict(test_dataset_En)
SVR_best_prediction_E

array([ 1.31135508e+00, -2.64038508e-01,  1.03062626e+00,  2.03871000e+00,
        4.18765685e-02,  2.06182877e-01,  1.72391982e-01,  9.61748743e-01,
       -4.88902652e-01,  1.27077325e-01,  2.52106215e+00,  1.19359030e+00,
       -2.59846379e-01,  1.20069911e+00,  4.97682442e-01, -4.22560554e-01,
        1.12559737e+00,  6.22440033e-02,  5.33909661e-01,  2.98452330e+00,
        1.03097548e+00,  8.45220104e-01,  1.49304162e+00,  8.78254606e-01,
        1.27601816e+00,  5.62939795e-02,  9.64038298e-01,  7.15363144e-01,
       -6.64953122e-02,  1.18666953e+00,  6.55362918e-01,  9.87276369e-01,
        8.59677983e-01,  1.10661490e+00,  2.12501656e-01,  9.96833076e-01,
       -1.10364998e-01,  2.60703107e-02,  2.01706651e+00,  6.22678144e-01,
        7.22909832e-01,  6.07047828e-04,  8.78179053e-01,  7.87860608e-01,
       -1.45972560e-02,  1.43114823e+00,  5.77188678e-01,  2.24361149e-01,
       -3.50801515e-01, -4.02608940e-01, -5.74331948e-01, -5.03291277e-01,
        3.15494855e-01,  

In [ ]:
#D.5.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [201]:
R2_SVR_best_E=r2_score(test_cible_En,SVR_best_prediction_E)
R2_SVR_best_E

0.5615509842747388

In [ ]:
#Calcul du MSE:

In [202]:
MSE_SVR_best_E=mean_squared_error(test_cible_En,SVR_best_prediction_E)

In [ ]:
#Calcul du RMSE:

In [203]:
RMSE_SVR_best_E=math.sqrt(MSE_SVR_best_E)

In [ ]:
#Calcul du MAE:

In [204]:
 MAE_SVR_best_E=mean_absolute_error(test_cible_En,SVR_best_prediction_E)

In [ ]:
#D.5.4 Résultats.

In [205]:
results_SVR_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_SVR_E= results_SVR_E.append({'Modèle':'SVR','R^2':R2_SVR_best_E,'MSE':MSE_SVR_best_E,'RMSE':RMSE_SVR_best_E,'MAE':MAE_SVR_best_E ,'Temps de calcul (s)':SVR_time_E.seconds},ignore_index=True)
results_SVR_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,SVR,0.561551,0.445636,0.66756,0.491549,220


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1 et de celle du random forest, mais plus proche que celles du dummy regressor, la
#  régression linéaire et le elasticNet.
# -la valeur du RMSE est supérieure à 0.5 et à de celle du random forest, mais plus proche que le dummy regressor, la
#  régression linéaire et le elasticNet.
# -La valeur du MAE est moins bonne celle du random forest, mais meilleure que celles du modèle naïf de dummy regressor, de 
#la régression linéaire et du elasticNet (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire, le elasticNet et le random forest.

# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques, mais vaut-il vraiment le coup pour
#cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?

#---------------------------------------------------------------------------------------------------------------------------


#D.6 Tableau résumé:


In [206]:
results_total_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_total_E= results_total_E.append({'Modèle':'Dummy Regressor','R^2':R2_DM_E,'MSE':MSE_DM_E,'RMSE':RMSE_DM_E,'MAE':MAE_DM_E ,'Temps de calcul (s)':DM_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'Linear Regression','R^2':R2_LR_E,'MSE':MSE_LR_E,'RMSE':RMSE_LR_E,'MAE':MAE_LR_E ,'Temps de calcul (s)':LR_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'ElasticNet','R^2':R2_EN_best_E,'MSE':MSE_EN_best_E,'RMSE':RMSE_EN_best_E,'MAE':MAE_EN_best_E ,'Temps de calcul (s)':EN_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'Random Forest','R^2':R2_RF_best_E,'MSE':MSE_RF_best_E,'RMSE':RMSE_RF_best_E,'MAE':MAE_RF_best_E,'Temps de calcul (s)':RF_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'SVR','R^2':R2_SVR_best_E,'MSE':MSE_SVR_best_E,'RMSE':RMSE_SVR_best_E,'MAE':MAE_SVR_best_E,'Temps de calcul (s)':SVR_time_E.seconds},ignore_index=True)
results_total_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.001851,1.018274,1.009096,0.778984,1
1,Linear Regression,0.392084,0.617882,0.786055,0.612551,1
2,ElasticNet,0.390408,0.619585,0.787137,0.613251,25
3,Random Forest,0.752303,0.251757,0.501754,0.358595,411
4,SVR,0.561551,0.445636,0.667560,0.491549,220


In [ ]:
#Montrer les deux tableaux ensembles:

In [207]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [208]:
display_side_by_side(results_total,results_total_E, titles=['Sans ENERGYSTARScore','Avec ENERGYSTARScore'])

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.004425,1.048740,1.024080,0.792805,1
1,Linear Regression,0.289287,0.742070,0.861435,0.682741,5
2,ElasticNet,0.212837,0.821893,0.906583,0.705609,14
3,Random Forest,0.756289,0.254463,0.504443,0.346645,337
4,SVR,0.396834,0.629777,0.793585,0.618638,457
,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.001851,1.018274,1.009096,0.778984,1
1,Linear Regression,0.392084,0.617882,0.786055,0.612551,1
2,ElasticNet,0.390408,0.619585,0.787137,0.613251,25
3,Random Forest,0.752303,0.251757,0.501754,0.358595,411


In [ ]:
#Ainsi, voyant les inconvénients niveau calcul et stabilité du 'ENERGYSTARScore' on préfère ne pas le considérer dans notre
#modèle car il représente plus d’inconvénients de calcul que d'apports bénéfiques dans nos modèle.
#Et bien sûr on reste avec le Random Forest.